Testing a slew of clustering algorithms

In [14]:
import pickle as pkl
from cnns.core import SOMClusterer as som
from cnns.core import APClusterer as ap
from cnns.utils import clustering_utils as clu
import numpy as np

In [15]:
def combine_clustering_algos(face_ids, X, clusterer_config_map):
    num_models = len(clusterer_config_map)
    cluster_id_map = np.empty((X.shape[0], num_models + 1))
    cluster_id_map[:, -1] = -1
    dfs = []
    for clusterer, config in clusterer_config_map.iteritems():
        c_ = clusterer(face_ids, X, **config)
        df = c_.run()
        df = df[['face_id', 'cluster_idx']]
        dfs.append(df)
    return dfs

def merge_multi_cluster_labels(self, cluster_id_map):
    unmerged_set = set(range(cluster_id_map.shape[0]))
    thresh = (cluster_id_map.shape[1] - 1)/2
    global_cluster_idx = 0
    while len(unmerged_set) > 0:
        curr_cluster = set([])
        i = list(unmerged_set)[0]
        cluster_id_map[i, -1] = global_cluster_idx
        curr_cluster.add(i)
        for j in unmerged_set:
            if i != j:
                num_same_cluster_id = len(
                    np.where((cluster_id_map[i, :-1] - cluster_id_map[j, :-1]) == 0)[0])

                # take the majority vote
                if num_same_cluster_id > thresh:
                    cluster_id_map[j, -1] = global_cluster_idx
                    curr_cluster.add(j)
        global_cluster_idx += 1
        unmerged_set = unmerged_set.difference(curr_cluster)

In [8]:
user_id = 2685009
base_dirpath = '/Users/babasarala/Desktop/face_clustering_full_data'
settings_ver = 1.2
comp_type = None
pickle_filepath = '%s/%i/%i_prod_%.1f_%s_cnn_codes.p'%(base_dirpath, user_id, user_id, settings_ver, comp_type)

In [9]:
img_urls, X = pkl.load(open(pickle_filepath, 'rb'))

In [5]:
som_config = {'grid_size':15, 'sigma':1.0, 'learning_rate':0.25, 'num_trials':3, 'num_iter':100, 'normalize':True}
ap_config = {'damping':0.5, 'convergence_iter':15, 'max_iter':200, 'affinity':'euclidean'}
clusterer_config_map = {som.SOMClusterer:som_config, ap.APClusterer:ap_config}

In [16]:
dfs = combine_clustering_algos(img_urls, X, clusterer_config_map)

KeyError: "['face_id'] not in index"